In [90]:
import os
import re
import pickle
import numpy as np
import pandas as pd

from collections import Counter

from sklearn.metrics import f1_score, make_scorer, confusion_matrix, \
    classification_report
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, \
    StratifiedShuffleSplit, RandomizedSearchCV

# Load data

In [91]:
with open("../Data/Learn/sentences.pkl", "rb") as f:
    learn_sentences = pickle.load(f)
    
with open("../Data/Test//sentences.pkl", "rb") as f:
    test_sentences = pickle.load(f)

In [3]:
words = []
with(open("../Data/wiki.fr.vec")) as f:
    for line in f:
        words.append(line.split()[0])
words.pop(0)
words_set = set(words)

'1152449'

# Preprocessing

In [35]:
# Train
my_prep_sentences = [re.sub("\s*([^\w\s])\s*", " \\1 ", x).strip().lower() for x in learn_sentences]
my_prep_sentences = [re.sub("\d+", " UNK ", x).strip().lower() for x in my_prep_sentences]
my_vocab = list(set([x for sent in my_prep_sentences for x in sent.split()]))
my_vocab = {x for x in my_vocab if x in words_set}
my_vocab = {x: i + 1 for i, x in enumerate(my_vocab)}
my_vocab["UNK"] = 0
my_reverse_vocab = {v: k for k, v in my_vocab.items()}
my_sequences = [[my_vocab.get(x, 0) for x in sent.split()] for sent in my_prep_sentences]
len(my_vocab), len(my_reverse_vocab)

(26709, 26709)

In [92]:
# Test
my_prep_test_sentences = [re.sub("\s*([^\w\s])\s*", " \\1 ", x).strip().lower() for x in test_sentences]
my_prep_test_sentences = [re.sub("\d+", " UNK ", x).strip().lower() for x in my_prep_test_sentences]
my_test_sequences = [[my_vocab.get(x, 0) for x in sent.split()] for sent in my_prep_test_sentences]

In [36]:
my_prep_sentences[:5]

["j ' aurai l ' occasion de dire aux français comment notre enseignement devra évoluer pour permettre à chaque jeune de trouver sa place , d ' entrer dans le monde du travail , de savoir s ' adapter et , à partir de là , d ' acquérir , tout au long de la vie , de nouvelles compétences et de nouveaux savoirs .",
 'il est nécessaire .',
 "dans votre coeur et dans votre vie , la confiance et l ' enthousiasme l ' emportent sur le doute .",
 "pour conduire ce débat dans un esprit de véritable dialogue , je compte , si nos partenaires en sont d ' accord , inviter au prochain sommet du g unk  , à lyon , pour une séance de travail , le secrétaire général des nations unies , le président de la banque mondiale et le directeur général du fonds monétaire international .",
 "la france et l ' europe construiront ainsi un avenir de coopération avec un proche - orient pacifié , stable , prospère , libéré des menaces de la guerre , de la prolifération , du terrorisme ."]

In [93]:
my_prep_test_sentences[:5]

['et tous se demandaient :  " après elle , après nous , qui se souviendra et comment ?',
 "qui feront de l ' union ce grand espace de paix , de droits et de libertés , ce foyer de l ' esprit digne de son héritage , cette terre que nos citoyens aimeront habiter , cultiver , faire rayonner ensemble .",
 "l ' enfant a été , naturellement , le premier bénéficiaire des victoires remportées contre la maladie et contre la malnutrition .",
 "loin d ' être isolés , ces exemples se multiplient .",
 "vous l ' avez souligné , monsieur le président , les professionnels libéraux sont au service de l ' homme ."]

# Load embeddings

In [49]:
embeddings = np.zeros((len(my_vocab), 301), dtype=np.float64)
found_words = set()
with(open("../Data/wiki.fr.vec")) as f:
    embeddings[0, -1] = 1
    for line in f:
        elements = line.split()
        index = my_vocab.get(elements[0], None)
        if index is None:
            continue
        found_words.add(elements[0])
        embeddings[index, :300] = np.fromiter((float(x) for x in elements[-300:]), np.float64, 300)

In [55]:
len(found_words) == len(my_vocab) - 1

True

In [63]:
embeddings[my_vocab["UNK"]], embeddings[my_vocab["bonjour"]]

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [96]:
if not os.path.exists("../Data/generated"):
    os.mkdir("../Data/generated")
with open("../Data/generated/my_learn_sequences.p", "wb") as f:
    pickle.dump(my_sequences, f)
with open("../Data/generated/my_test_sequences.p", "wb") as f:
    pickle.dump(my_test_sequences, f)
with open("../Data/generated/my_vocab.p", "wb") as f:
    pickle.dump(my_vocab, f)
with open("../Data/generated/my_embeddings.p", "wb") as f:
    pickle.dump(embeddings, f)